### Clean Merge Result

In [ ]:
# !pip install modin[ray]

In [ ]:
import ray  # ayrı olarak çalıştırılmalı
ray.init() 
# Not: Eğer multiprocessing kullanılacaksa ray init kullanılmamalı

In [1]:
#import modin.pandas as pd
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', 50)

#### Clean One Character And Digit

In [2]:
df = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Sentence/Spark Turkish Sentence Merge/Clean_Spark_Sentence_Merge_Result.csv")
df = df.iloc[:,0:2]
df

,sentence,frequency
0,evet,1948596
1,hayır,1250401
2,tamam,882921
3,ne,753105
4,teşekkürler,343527
...,...,...
20975605,omalley bize buzsuz scotch ver,2
20975606,omalley bu acele ne,2
20975607,omalley bu milton wing,2
20975608,omalley buna cevap verecek zamanım,2


In [3]:
df = df[~df.iloc[:,0].str.contains(fr"fff", na=True)]
df.reset_index(drop=True, inplace=True)
df

,sentence,frequency
0,evet,1948596
1,hayır,1250401
2,tamam,882921
3,ne,753105
4,teşekkürler,343527
...,...,...
20972955,omalley bize buzsuz scotch ver,2
20972956,omalley bu acele ne,2
20972957,omalley bu milton wing,2
20972958,omalley buna cevap verecek zamanım,2


In [5]:
def clean_num(var):
    var_list = re.findall("\w+", var)
    try:
        if var_list[0][0].isdigit() or (len(var_list) <= 1 and len(var_list[0]) <= 1):
            return "digit or one character"
        else:
            return var
    except:
        return "not digit or character"

In [6]:
df.iloc[:,0] = df.iloc[:,0].apply(lambda x : clean_num(x))
df

,sentence,frequency
0,evet,1948596
1,hayır,1250401
2,tamam,882921
3,ne,753105
4,teşekkürler,343527
...,...,...
20972955,omalley bize buzsuz scotch ver,2
20972956,omalley bu acele ne,2
20972957,omalley bu milton wing,2
20972958,omalley buna cevap verecek zamanım,2


In [7]:
df[df.iloc[:,0] == "digit or one character"]

,sentence,frequency
175,digit or one character,27460
1008,digit or one character,5365
1079,digit or one character,5066
1150,digit or one character,4782
1286,digit or one character,4331
...,...,...
16110075,digit or one character,2
16110241,digit or one character,2
16110265,digit or one character,2
16110350,digit or one character,2


In [8]:
df[df.iloc[:,0] == "not digit or character"]

,sentence,frequency


In [9]:
df.drop(df[df.iloc[:,0] == "digit or one character"].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,sentence,frequency
0,evet,1948596
1,hayır,1250401
2,tamam,882921
3,ne,753105
4,teşekkürler,343527
...,...,...
20776151,omalley bize buzsuz scotch ver,2
20776152,omalley bu acele ne,2
20776153,omalley bu milton wing,2
20776154,omalley buna cevap verecek zamanım,2


In [10]:
# df.drop(df[df.iloc[:,0] == "not digit or character"].index, inplace=True)
# df.reset_index(drop=True, inplace=True)
# df

In [11]:
df.to_csv("Merge_Result_Clean_Digit_One_Char.csv", index=False)

#### Include Name Entity

In [4]:
df_no_digit = pd.read_csv("Merge_Result_Clean_Digit_One_Char.csv")
df_no_digit = df_no_digit.iloc[:,0:2].head(100000)
df_no_digit

,sentence,frequency
0,evet,1948596
1,hayır,1250401
2,tamam,882921
3,ne,753105
4,teşekkürler,343527
...,...,...
99995,şahitlerim var,67
99996,genç ve aptaldım,67
99997,biliyor musun bilmiyor musun,67
99998,kadınmış,67


In [5]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re

In [6]:
help(multiprocessing)

Help on package multiprocessing:

NAME
    multiprocessing

MODULE REFERENCE
    https://docs.python.org/3.9/library/multiprocessing
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    # Package analogous to 'threading.py' but using processes
    #
    # multiprocessing/__init__.py
    #
    # This package is intended to duplicate the functionality (and much of
    # the API) of threading.py but uses processes instead of threads.  A
    # subpackage 'multiprocessing.dummy' has the same API but is a simple
    # wrapper for 'threading'.
    #
    # Copyright (c) 2006-2008, R Oudkerk
    # Licensed to PSF under a Contributor Agreement.
    #

PACKAGE CONTENTS
    connection
    context
    dummy (packag

In [7]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [8]:
df_ne = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Name Entity/Special_Name_Unique.xlsx")
df_ne = df_ne.head(10)
df_ne

,spec_name
0,aaron
1,abbas
2,abbey
3,abby
4,abe
5,abigaile
6,abraham
7,absinthe
8,abubakar
9,achilles


In [9]:
ne_list = [x.lower() for x in df_ne.iloc[:,0]]

In [10]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
special_name_list = manager.list()

def find_special_name(name_ent):
    name_in_df = df_no_digit[df_no_digit.iloc[:,0].str.contains(fr"(?:\s|^){name_ent}(?:\s|$)", na=True)]
    ne_list = name_in_df.iloc[:,0].to_list()
    for j in ne_list:
        special_name_list.append(j) 

if __name__ == '__main__':
    with Pool(8) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(find_special_name, ne_list) # string_word liste

In [11]:
list_special_name = list(special_name_list)
df_special_name = pd.DataFrame(list_special_name)
df_special_name.rename(columns={0:"sentence"}, inplace=True)
#df_special_name.rename(columns={0:"word"}, inplace=True)
df_special_name

,sentence
0,aaron
1,merhaba aaron
2,selam aaron
3,aaron nerede
4,abbey
5,abe
6,abraham
7,abraham lincoln
8,abby
9,ben abby


In [12]:
df_special_freq = pd.merge(df_no_digit, df_special_name, how="inner", on="sentence")
#df_special_freq = pd.merge(df_no_digit, df_special_name, how="inner", on="word")
df_special_freq

,sentence,frequency
0,abby,3842
1,aaron,2235
2,achilles,931
3,abe,709
4,abraham,629
5,abraham lincoln,233
6,abbey,233
7,ben abby,149
8,merhaba abby,148
9,merhaba aaron,125


In [22]:
df_special_freq.to_csv("Include_Name_Entity_Sentence.csv", index=False)

#### Extract Name Entity From Data

In [13]:
df_no_digit = pd.read_csv("Merge_Result_Clean_Digit_One_Char.csv")
df_no_digit = df_no_digit.iloc[:,0:2].head(100000)
#df_no_digit = df_no_digit[["sentence"]].head(1000000)
df_no_digit

,sentence,frequency
0,evet,1948596
1,hayır,1250401
2,tamam,882921
3,ne,753105
4,teşekkürler,343527
...,...,...
99995,şahitlerim var,67
99996,genç ve aptaldım,67
99997,biliyor musun bilmiyor musun,67
99998,kadınmış,67


In [17]:
clean_set = set(df_no_digit.iloc[:,0])

In [18]:
df_special_freq = pd.read_csv("Include_Name_Entity_Result.csv")
df_special_freq

,sentence,frequency
0,abby,3842
1,aaron,2235
2,achilles,931
3,abe,709
4,abraham,629
5,abraham lincoln,233
6,abbey,233
7,ben abby,149
8,merhaba abby,148
9,merhaba aaron,125


In [19]:
name_set = set(df_special_name.iloc[:,0])

In [20]:
df_new = pd.DataFrame(clean_set.difference(name_set))
df_new.rename(columns={0:"sentence"}, inplace=True)
#df_new.rename(columns={0:"word"}, inplace=True)
df_new.reset_index(drop=True, inplace=True)
df_new

,sentence
0,pekâlâ efendim
1,corky
2,üzgünüm hayır
3,yüce sezar
4,koçum
...,...
99980,baba hadi
99981,ne var biliyor musun
99982,çok demode
99983,bırak tahmin edeyim


In [21]:
df_new_freq = pd.merge(df_no_digit, df_new, how="inner", on="sentence")
#df_new_freq = pd.merge(df_no_digit, df_new, how="inner", on="word")
df_new_freq

,sentence,frequency
0,evet,1948596
1,hayır,1250401
2,tamam,882921
3,ne,753105
4,teşekkürler,343527
...,...,...
99980,şahitlerim var,67
99981,genç ve aptaldım,67
99982,biliyor musun bilmiyor musun,67
99983,kadınmış,67


In [22]:
df_new_freq.to_csv("Merge_Result_Without_Name_Entity.csv", index=False)

#### Name Entity Change Same Name

In [23]:
import ray  # ayrı olarak çalıştırılmalı
ray.init() 

2021-12-12 17:40:45,309	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.37',
 'raylet_ip_address': '192.168.1.37',
 'redis_address': '192.168.1.37:6379',
 'object_store_address': '/tmp/ray/session_2021-12-12_17-40-42_230353_116091/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-12-12_17-40-42_230353_116091/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-12-12_17-40-42_230353_116091',
 'metrics_export_port': 50921,
 'node_id': 'ca3cb0b4c46d31353d5ab520daffb9facb754fdfd7a4b5f48d95a6b2'}

In [24]:
import modin.pandas as pd

In [25]:
df_special_freq = pd.read_csv("Include_Name_Entity_Result.csv")
df_special_freq

,sentence,frequency
0,abby,3842
1,aaron,2235
2,achilles,931
3,abe,709
4,abraham,629
5,abraham lincoln,233
6,abbey,233
7,ben abby,149
8,merhaba abby,148
9,merhaba aaron,125


In [26]:
df_ne = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/Result/Name Entity/Special_Name_Unique.xlsx")
df_ne = df_ne.head(10)
df_ne

,spec_name
0,aaron
1,abbas
2,abbey
3,abby
4,abe
5,abigaile
6,abraham
7,absinthe
8,abubakar
9,achilles


In [27]:
ne_list = [x.lower() for x in df_ne.iloc[:,0]]

In [28]:
def replace_name(x):
    for i in ne_list:
        #x = re.sub(fr"(?:\s|^){i}", " ali ", x)
        x = re.sub(fr"(?:\s|^){i}(?:\s|$)", " fikret ", x)
        x = x.strip()        
    return x

In [29]:
df_special_freq.iloc[:,0] = df_special_freq.iloc[:,0].apply(lambda x : replace_name(x))
df_special_freq.drop_duplicates(inplace=True)
df_special_freq

,sentence,frequency
0,fikret,3842
1,fikret,2235
2,fikret,931
3,fikret,709
4,fikret,629
5,fikret lincoln,233
6,fikret,233
7,ben fikret,149
8,merhaba fikret,148
9,merhaba fikret,125


In [30]:
df_spec_sum = df_special_freq.groupby(df_special_freq.iloc[:,0])[["frequency"]].sum()
df_spec_sum = df_spec_sum.sort_values(by="frequency", ascending=False)
df_spec_sum.reset_index(inplace=True)
df_spec_sum

,sentence,frequency
0,fikret,8579
1,merhaba fikret,273
2,fikret lincoln,233
3,fikret nerede,193
4,selam fikret,163
5,ben fikret,149
6,hey fikret,74


In [31]:
df_spec_sum.to_csv("Name_Entity_Change_Same.csv", index=False)

#### Concat Without Name Entity And Change Same

In [32]:
df_new_freq= pd.read_csv("Merge_Result_Without_Name_Entity.csv")
df_new_freq

,sentence,frequency
0,evet,1948596
1,hayır,1250401
2,tamam,882921
3,ne,753105
4,teşekkürler,343527
...,...,...
99980,şahitlerim var,67
99981,genç ve aptaldım,67
99982,biliyor musun bilmiyor musun,67
99983,kadınmış,67


In [33]:
df_spec_sum = pd.read_csv("Name_Entity_Change_Same.csv")
df_spec_sum

,sentence,frequency
0,fikret,8579
1,merhaba fikret,273
2,fikret lincoln,233
3,fikret nerede,193
4,selam fikret,163
5,ben fikret,149
6,hey fikret,74


In [34]:
df_clean_merge_result = pd.concat([df_new_freq, df_spec_sum], axis=0)
df_clean_merge_result = df_clean_merge_result.sort_values(by="frequency", ascending=False)
df_clean_merge_result

,sentence,frequency
0,evet,1948596
1,hayır,1250401
2,tamam,882921
3,ne,753105
4,teşekkürler,343527
...,...,...
98975,merhaba lisbon,67
98974,benimle de,67
98973,rictus,67
98972,canım yanacak mı,67


In [ ]:
df_clean_merge_result.to_csv("Clean_Spark_Sentence_Merge_Result2.csv", index=False)